In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd



In [29]:
#test data
dates = pd.date_range(start="2024-01-01", periods=100, freq="D")
closing_prices = [
    101.764052, 100.602177, 101.382778, 102.846954, 102.675639,
    100.576772, 103.376212, 102.380072, 102.850681, 103.727309,
    104.455994, 103.413931, 104.323544, 105.386031, 104.269430,
    105.688740, 105.870469, 106.424284, 106.281054, 107.322950,
    108.312516, 107.793377, 108.333080, 108.977315, 109.477676,
    109.235864, 110.550167, 110.498905, 110.775643, 111.303888,
    110.764702, 112.131915, 112.121944, 112.422042, 113.130003,
    113.177382, 113.281641, 113.836941, 113.670423, 115.197978,
    115.261336, 115.026351, 115.595684, 116.109046, 116.251534,
    116.531812, 117.491118, 117.154489, 117.893415, 118.606026,
    118.724007, 118.643595, 119.046720, 118.774653, 119.396953,
    119.724659, 120.198746, 120.167993, 120.883129, 120.503661,
    120.883185, 121.676492, 121.136197, 121.598367, 122.359674,
    122.086220, 122.489546, 122.703341, 123.304508, 123.361046,
    123.562961, 124.037057, 124.464109, 124.135171, 124.675325,
    125.276172, 125.355014, 125.710053, 126.379586, 126.072420,
    126.525589, 126.674861, 126.999894, 127.737785, 127.296859,
    127.979930, 128.236258, 128.874720, 128.768569, 129.456712,
    129.296263, 129.854942, 130.375448, 130.632321, 130.816212,
    131.517870, 131.374998, 132.230947, 132.643171, 132.922100,
    133.494745, 133.524597, 134.345049, 134.269553
][:100]  # Trim to 100 entries

# Create the DataFrame with matching lengths
your_stock_data = pd.DataFrame({
    "Date": dates,
    "Close": closing_prices
})

# Display the first few rows to confirm structure
your_stock_data.head()

,Date,Close
0,2024-01-01,101.764052
1,2024-01-02,100.602177
2,2024-01-03,101.382778
3,2024-01-04,102.846954
4,2024-01-05,102.675639


In [30]:
# Scale data
# Separate the 'Close' column for scaling
close_prices = your_stock_data[['Close']].values  # Extract 'Close' column as a 2D array

# Initialize and apply MinMaxScaler to the 'Close' prices
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close_prices = scaler.fit_transform(close_prices)

# If you need to put the data back into the DataFrame
scaled_stock_data = your_stock_data.copy()
scaled_stock_data['Close'] = scaled_close_prices

# Display the first few rows to confirm scaling
scaled_stock_data.head()

,Date,Close
0,2024-01-01,0.036706
1,2024-01-02,0.000785
2,2024-01-03,0.024919
3,2024-01-04,0.070186
4,2024-01-05,0.064889


In [31]:
# Prepare training data with a 60-day look-back
X_train, y_train = [], []
for i in range(60, len(scaled_stock_data)):
    # Use iloc to select the 'Close' values in a 60-day window
    X_train.append(scaled_stock_data['Close'].iloc[i-60:i].values)  # 60-day look-back
    y_train.append(scaled_stock_data['Close'].iloc[i])

# Convert lists to numpy arrays for model compatibility
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape X_train to 3D for LSTM input (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_train.shape, y_train.shape  # Check shapes for compatibility


((40, 60, 1), (40,))

In [32]:
# Define LSTM model
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(units=50, return_sequences=False),
    Dropout(0.2),
    Dense(units=25),
    Dense(units=1)
])

2024-11-01 09:38:39.238144: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-01 09:38:39.239210: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-01 09:38:39.240235: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [35]:

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions on the training set
predictions = model.predict(X_train)

# Inverse transform predictions if the data was scaled (assuming 'scaler' was used for MinMax scaling)
predicted_prices = scaler.inverse_transform(predictions)

# Display the predicted stock prices
print("Predicted Stock Prices:\n", predicted_prices[:10])  # Displaying the first 10 predictions for brevity


Epoch 1/10


2024-11-01 09:40:24.818798: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-01 09:40:24.819935: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-01 09:40:24.820798: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2/2 [==============================] - 3s 54ms/step - loss: 0.0102
Epoch 2/10
2/2 [==============================] - 0s 51ms/step - loss: 0.0089
Epoch 3/10
2/2 [==============================] - 0s 39ms/step - loss: 0.0055
Epoch 4/10
2/2 [==============================] - 0s 46ms/step - loss: 0.0049
Epoch 5/10
2/2 [==============================] - 0s 40ms/step - loss: 0.0053
Epoch 6/10
2/2 [==============================] - 0s 48ms/step - loss: 0.0066
Epoch 7/10
2/2 [==============================] - 0s 57ms/step - loss: 0.0066
Epoch 8/10
2/2 [==============================] - 0s 59ms/step - loss: 0.0061
Epoch 9/10
2/2 [==============================] - 0s 54ms/step - loss: 0.0084
Epoch 10/10
2/2 [==============================] - 0s 45ms/step - loss: 0.0072


2024-11-01 09:40:28.791367: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-01 09:40:28.792362: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-01 09:40:28.793399: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2/2 [==============================] - 1s 15ms/step
Predicted Stock Prices:
 [[121.864235]
 [122.06338 ]
 [122.27516 ]
 [122.46701 ]
 [122.652115]
 [122.85078 ]
 [123.040276]
 [123.2286  ]
 [123.41473 ]
 [123.61179 ]]
